## Getting Kaggle's Data

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

In [ ]:
data = pd.read_csv("train.csv")

In [ ]:
data.columns

In [ ]:
data.isna().sum()

In [ ]:
data_fixed = data.copy()

## Cleaning Periodo Column

La función cleaning_periodo procesa el DataFrame al descomponer la columna PERIODO, que contiene datos en un formato que combina año y bimestre, en dos nuevas columnas: AÑO y BIMESTRE. Extrae los primeros cuatro caracteres de PERIODO para crear AÑO y el quinto carácter para crear BIMESTRE, ambos convertidos a tipo numérico. Finalmente, elimina la columna original PERIODO, dejando el DataFrame más limpio y estructurado para análisis posteriores.

In [ ]:
def cleaning_periodo(data_fixed):
  data_fixed['AÑO'] = pd.to_numeric(data_fixed['PERIODO_ACADEMICO'].astype(str).str[:4])
  data_fixed['BIMESTRE'] = pd.to_numeric(data_fixed['PERIODO_ACADEMICO'].astype(str).str[4])
  data_fixed.drop(columns=['PERIODO_ACADEMICO'], inplace=True)

In [ ]:
cleaning_periodo(data_fixed)
data_fixed

# Limpieza de datos

In [ ]:
def cleaning_programa_academico(data_fixed):
  freq_encoding = data_fixed['E_PRGM_ACADEMICO'].value_counts(normalize=True)
  data_fixed['E_PRGM_ACADEMICO'] = data_fixed['E_PRGM_ACADEMICO'].map(freq_encoding)

In [ ]:
def cleaning_departamento(data_fixed):
  freq_encoding = data_fixed['E_PRGM_DEPARTAMENTO'].value_counts(normalize=True)
  data_fixed['E_PRGM_DEPARTAMENTO'] = data_fixed['E_PRGM_DEPARTAMENTO'].map(freq_encoding)

In [ ]:
def cleaning_matricula(data_fixed):
    valores_matricula = {
        'Entre 5.5 millones y menos de 7 millones': 6.25,
        'Entre 2.5 millones y menos de 4 millones': 3.25,
        'Entre 4 millones y menos de 5.5 millones': 4.75,
        'Más de 7 millones': 7.0,
        'Entre 1 millón y menos de 2.5 millones': 1.75,
        'Entre 500 mil y menos de 1 millón': 0.75,
        'Menos de 500 mil': 0.25,
        'No pagó matrícula': 0,
    }
    data_fixed['E_VALORMATRICULAUNIVERSIDAD'] = data_fixed['E_VALORMATRICULAUNIVERSIDAD'].map(valores_matricula)
    mean = data_fixed['E_VALORMATRICULAUNIVERSIDAD'].mean()
    std = data_fixed['E_VALORMATRICULAUNIVERSIDAD'].std()
    sample = np.random.normal(mean, std, data_fixed['E_VALORMATRICULAUNIVERSIDAD'].isna().sum())
    data_fixed.loc[data_fixed['E_VALORMATRICULAUNIVERSIDAD'].isna(), 'E_VALORMATRICULAUNIVERSIDAD'] = sample

In [ ]:
def cleaning_horas_trabajo(data_fixed):
    horas_promedio = {
        'Menos de 10 horas': 5,
        '0': 0,
        'Más de 30 horas': 35,
        'Entre 21 y 30 horas': (21 + 30) / 2,
        'Entre 11 y 20 horas': (11 + 20) / 2
    }
    data_fixed['E_HORASSEMANATRABAJA'] = data_fixed['E_HORASSEMANATRABAJA'].map(horas_promedio)
    mean = data_fixed['E_HORASSEMANATRABAJA'].mean()
    std = data_fixed['E_HORASSEMANATRABAJA'].std()
    sample = np.random.normal(mean, std, data_fixed['E_HORASSEMANATRABAJA'].isna().sum())
    data_fixed.loc[data_fixed['E_HORASSEMANATRABAJA'].isna(), 'E_HORASSEMANATRABAJA'] = sample

In [ ]:
def cleaning_estrato(data_fixed):
  # Rellena los NA con el promedio (como cadena)
  label_encoder = LabelEncoder()
  data_fixed['F_ESTRATOVIVIENDA'] = label_encoder.fit_transform(data_fixed['F_ESTRATOVIVIENDA'])

In [ ]:
def cleaning_internet(data_fixed):
    data_fixed['F_TIENEINTERNET'] = data_fixed['F_TIENEINTERNET'].fillna('Sin Información')
    data_fixed['F_TIENEINTERNET'] = data_fixed['F_TIENEINTERNET'].map({
        'Si': 1,
        'No': 0,
        'Sin Información': -1
    })

In [ ]:
def cleaning_padre(data_fixed):
    data_fixed['F_EDUCACIONPADRE'] = data_fixed['F_EDUCACIONPADRE'].fillna('Sin Información')
    data_fixed['F_EDUCACIONPADRE'] = data_fixed['F_EDUCACIONPADRE'].replace({
        'No sabe': -1, 'No Aplica': -1, 'Sin Información': -1
    })
    levels = ['Ninguno', 'Primaria incompleta', 'Primaria completa',
              'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa',
              'Técnica o tecnológica incompleta', 'Técnica o tecnológica completa',
              'Educación profesional incompleta', 'Educación profesional completa',
              'Postgrado']
    mask = data_fixed['F_EDUCACIONPADRE'] != -1
    encoder = OrdinalEncoder(categories=[levels])
    data_fixed.loc[mask, 'F_EDUCACIONPADRE'] = encoder.fit_transform(data_fixed.loc[mask, ['F_EDUCACIONPADRE']])
    data_fixed['F_EDUCACIONPADRE'] = data_fixed['F_EDUCACIONPADRE'].astype(int)

In [ ]:
def cleaning_madre(data_fixed):
    data_fixed['F_EDUCACIONMADRE'] = data_fixed['F_EDUCACIONMADRE'].fillna('Sin Información')
    data_fixed['F_EDUCACIONMADRE'] = data_fixed['F_EDUCACIONMADRE'].replace({
        'No sabe': -1, 'No Aplica': -1, 'Sin Información': -1
    })
    levels = ['Ninguno', 'Primaria incompleta', 'Primaria completa',
              'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa',
              'Técnica o tecnológica incompleta', 'Técnica o tecnológica completa',
              'Educación profesional incompleta', 'Educación profesional completa',
              'Postgrado']
    mask = data_fixed['F_EDUCACIONMADRE'] != -1
    encoder = OrdinalEncoder(categories=[levels])
    data_fixed.loc[mask, 'F_EDUCACIONMADRE'] = encoder.fit_transform(data_fixed.loc[mask, ['F_EDUCACIONMADRE']])
    data_fixed['F_EDUCACIONMADRE'] = data_fixed['F_EDUCACIONMADRE'].astype(int)



In [ ]:
def cleaning_pago(data_fixed):
    data_fixed['E_PAGOMATRICULAPROPIO'] = data_fixed['E_PAGOMATRICULAPROPIO'].fillna('Sin Información')
    data_fixed['E_PAGOMATRICULAPROPIO'] = data_fixed['E_PAGOMATRICULAPROPIO'].map({
        'Si': 1,
        'No': 0,
        'Sin Información': -1
    })

In [ ]:
def cleaning_rendimiento(data_fixed):
  rmap = {'alto': 3, 'bajo':0, 'medio-bajo':1, 'medio-alto':2}
  data_fixed['RENDIMIENTO_GLOBAL'] = data_fixed['RENDIMIENTO_GLOBAL'].map(rmap)

## Function with the complete cleaning

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

def data_cleaning(data):
    data_fixed = data.copy()
    cleaning_periodo(data_fixed)
    cleaning_programa_academico(data_fixed)
    cleaning_departamento(data_fixed)
    cleaning_matricula(data_fixed)
    cleaning_horas_trabajo(data_fixed)
    cleaning_estrato(data_fixed)
    cleaning_internet(data_fixed)
    cleaning_padre(data_fixed)
    cleaning_madre(data_fixed)
    cleaning_pago(data_fixed)
    if 'RENDIMIENTO_GLOBAL' in data_fixed.columns:
      cleaning_rendimiento(data_fixed)
    return data_fixed


In [ ]:
data_fixed = data_cleaning(data)
data_fixed

In [ ]:
data_fixed.isna().sum()

In [ ]:
data_fixed.to_csv("data.csv", index=False)

# Model test


In [ ]:
data_fixed['PROMEDIO_EDUCACION_PADRES'] = (data_fixed["F_EDUCACIONMADRE"]+data_fixed['F_EDUCACIONPADRE'])/2

In [ ]:
from sklearn.model_selection import train_test_split
X = data_fixed.drop(columns=["RENDIMIENTO_GLOBAL", "ID",'AÑO',"BIMESTRE",'F_EDUCACIONPADRE','F_EDUCACIONMADRE'], axis=1)
y = data_fixed['RENDIMIENTO_GLOBAL']
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
!pip install lightgbm

In [ ]:
def cleaning_lavadora(data_fixed):
    data_fixed['F_TIENELAVADORA'] = data_fixed['F_TIENELAVADORA'].fillna('Sin Información')
    data_fixed['F_TIENELAVADORA'] = data_fixed['F_TIENELAVADORA'].map({
        'Si': 1,
        'No': 0,
        'Sin Información': -1
    })

In [ ]:
def cleaning_automovil(data_fixed):
    data_fixed['F_TIENEAUTOMOVIL'] = data_fixed['F_TIENEAUTOMOVIL'].fillna('Sin Información')
    data_fixed['F_TIENEAUTOMOVIL'] = data_fixed['F_TIENEAUTOMOVIL'].map({
        'Si': 1,
        'No': 0,
        'Sin Información': -1
    })

In [ ]:
def cleaning_privado_libertad(data_fixed):
    data_fixed['E_PRIVADO_LIBERTAD'] = data_fixed['E_PRIVADO_LIBERTAD'].fillna('Sin Información')
    data_fixed['E_PRIVADO_LIBERTAD'] = data_fixed['E_PRIVADO_LIBERTAD'].map({
        'Si': 1,
        'No': 0,
        'Sin Información': -1
    })

In [ ]:
def cleaning_computador(data_fixed):
    data_fixed['F_TIENECOMPUTADOR'] = data_fixed['F_TIENECOMPUTADOR'].fillna('Sin Información')
    data_fixed['F_TIENECOMPUTADOR'] = data_fixed['F_TIENECOMPUTADOR'].map({
        'Si': 1,
        'No': 0,
        'Sin Información': -1
    })

In [ ]:
def cleaning_internet_1(data_fixed):
    data_fixed['F_TIENEINTERNET.1'] = data_fixed['F_TIENEINTERNET.1'].fillna('Sin Información')
    data_fixed['F_TIENEINTERNET.1'] = data_fixed['F_TIENEINTERNET.1'].map({
        'Si': 1,
        'No': 0,
        'Sin Información': -1
    })

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

def data_cleaning(data):
    data_fixed = data.copy()
    cleaning_periodo(data_fixed)
    cleaning_programa_academico(data_fixed)
    cleaning_departamento(data_fixed)
    cleaning_matricula(data_fixed)
    cleaning_horas_trabajo(data_fixed)
    cleaning_estrato(data_fixed)
    cleaning_internet(data_fixed)
    cleaning_padre(data_fixed)
    cleaning_madre(data_fixed)
    cleaning_pago(data_fixed)
    cleaning_lavadora(data_fixed)
    cleaning_automovil(data_fixed)
    cleaning_privado_libertad(data_fixed)
    cleaning_computador(data_fixed)
    cleaning_internet_1(data_fixed)
    if 'RENDIMIENTO_GLOBAL' in data_fixed.columns:
      cleaning_rendimiento(data_fixed)
    return data_fixed

In [ ]:
from sklearn.model_selection import train_test_split
X = data_fixed.drop(columns=["RENDIMIENTO_GLOBAL", "ID",'AÑO',"BIMESTRE",'F_EDUCACIONPADRE','F_EDUCACIONMADRE'], axis=1)
y = data_fixed['RENDIMIENTO_GLOBAL']
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
model = LGBMClassifier(
    n_estimators=100,         # Número de iteraciones de boosting (número de árboles)
    learning_rate=0.1,        # Tasa de aprendizaje
    max_depth=-1,             # Profundidad máxima de los árboles
    num_leaves=31,            # Número máximo de hojas en cada árbol
    boosting_type='gbdt'
)
model.fit(X_train, y_train)

In [ ]:
y_preds_train = model.predict(X_train)
accuracy = accuracy_score(y_train, y_preds_train)
print(f'Precisión: {accuracy * 100:.2f}%')

In [ ]:
y_preds_test = model.predict(X_test)
accuracy = accuracy_score(y_test, y_preds_test)
print(f'Precisión: {accuracy * 100:.2f}%')

# Preparing Submission

In [ ]:
test = pd.read_csv("test.csv")

In [ ]:
data_fixed=data_cleaning(test)

In [ ]:
data_fixed['PROMEDIO_EDUCACION_PADRES'] = (data_fixed["F_EDUCACIONMADRE"]+data_fixed['F_EDUCACIONPADRE'])/2
data_fixed=data_fixed.drop(columns=["Unnamed: 0","ID","AÑO","BIMESTRE",'F_EDUCACIONPADRE','F_EDUCACIONMADRE'])

In [ ]:
y_preds_test = model.predict(data_fixed)

In [ ]:
def reverse_rendimiento(sumbmission):
    # Mapeo inverso: de números a etiquetas de rendimiento
    reverse_rmap = {3: 'alto', 0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto'}

    # Aplicar el mapeo inverso a la columna RENDIMIENTO_GLOBAL
    sumbmission['RENDIMIENTO_GLOBAL'] = sumbmission['RENDIMIENTO_GLOBAL'].map(reverse_rmap)

In [ ]:
print(y_preds_test)

In [ ]:
submission = pd.DataFrame([test["ID"].astype(str), y_preds_test], index=['ID', 'RENDIMIENTO_GLOBAL']).T
reverse_rendimiento(submission)
submission.dropna(inplace=True)


In [ ]:
submission

In [ ]:
submission.to_csv("my_submission.csv", index=False)
!head my_submission.csv